## Process data from NCAR Mesa Lab for ATOC4500 Data Science - 2021
written by: Jennifer.E.Kay@colorado.edu
last updated: Feb. 13, 2022

Original daily netcdf data for 2021 in .nc format from ftp://ftp.eol.ucar.edu/pub/archive/weather  
No days missing

Procssed here to:
1) make data continuous, add missing value when no data present (-999)
2) put all data into a single file

In [1]:
#import modules
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np   # basic math library  you will type np.$STUFF  e.g., np.cos(1)
import numpy.linalg as LA
from matplotlib.gridspec import GridSpec
import timeit
import datetime
import scipy.stats as stats
import glob
from datetime import date

In [2]:
### set up and testing
print(date.today())

## read in an example file -- check on the contents
filedir='mesa/'
filename='mlab.20210101.nc'
ds_example=xr.open_dataset(filedir+filename)
print(ds_example)

#print(ds_example['wspd'].shape)
#print(ds_example['time'])
#plt.plot(ds_example['wspd']);

print(ds_example['wspd'][0:289].shape)

2022-02-13
<xarray.Dataset>
Dimensions:      (time: 289)
Coordinates:
  * time         (time) datetime64[ns] 2021-01-01 ... 2021-01-02
Data variables:
    lat          float64 ...
    lon          float64 ...
    alt          float64 ...
    pres         (time) float64 ...
    rain_accum   (time) float64 ...
    rh           (time) float64 ...
    tdry         (time) float64 ...
    wdir         (time) float64 ...
    wspd         (time) float64 ...
    wspd_max     (time) float64 ...
    U            (time) float64 ...
    V            (time) float64 ...
    dewpoint     (time) float64 ...
    cpres0       (time) float64 ...
    windchill    (time) float64 ...
    raina_event  (time) float64 ...
    raina_daily  (time) float64 ...
Attributes:
    site_name:      NCAR Mesa Lab
    site_location:  NCAR Mesa Lab, South West Boulder, CO
    Conventions:    CF-1.7
(289,)


In [3]:
### here I read in all of the nc files for 2021

filedir='mesa/'

files=sorted(glob.glob(filedir+'mlab.2021*.nc'))
#files2021=sorted(glob.glob(filedir+'mlab.202105*.nc')) ## this is just may, used for testing
#files2021=files[120:151] ## this is just may, used for testing

#print(files)
print(type(files))
print(f'number of files: {len(files)}')

<class 'list'>
number of files: 365


In [4]:
### loop to check the dimensions of the files, determine when and how many have missing vales
count=0
for i in np.arange(0,len(files)):
    foo=xr.open_dataset(files[i])
    if (len(foo['wspd_max'])!=289):
        print(files[i])
        #print(len(foo['wspd_max']))
        count=count+1        
print(f'# of days with missing values: {count}') 

mesa/mlab.20210107.nc
mesa/mlab.20210112.nc
mesa/mlab.20210113.nc
mesa/mlab.20210114.nc
mesa/mlab.20210120.nc
mesa/mlab.20210121.nc
mesa/mlab.20210226.nc
mesa/mlab.20210313.nc
mesa/mlab.20210314.nc
mesa/mlab.20210407.nc
mesa/mlab.20210409.nc
mesa/mlab.20210416.nc
mesa/mlab.20210429.nc
mesa/mlab.20210503.nc
mesa/mlab.20210522.nc
mesa/mlab.20210523.nc
mesa/mlab.20210604.nc
mesa/mlab.20210618.nc
mesa/mlab.20210707.nc
mesa/mlab.20210723.nc
mesa/mlab.20210828.nc
mesa/mlab.20210829.nc
mesa/mlab.20210901.nc
mesa/mlab.20210902.nc
mesa/mlab.20210916.nc
mesa/mlab.20211006.nc
mesa/mlab.20211007.nc
mesa/mlab.20211008.nc
mesa/mlab.20211028.nc
mesa/mlab.20211029.nc
mesa/mlab.20211105.nc
mesa/mlab.20211214.nc
mesa/mlab.20211230.nc
# of days with missing values: 33


In [5]:
### loop over all days  when there is a missing data file - deal with it!
### fill in all missing values as -999
pres_all=[]
tdry_all=[]
rh_all=[]
wdir_all=[]
wspd_all=[]
wspdmax_all=[]
raina_all=[]

count=0
for i in np.arange(0,len(files)):
    print(f'working on {files[i]}')
    foo=xr.open_dataset(files[i])
    pres=foo['pres'].values
    tdry=foo['tdry'].values
    rh=foo['rh'].values
    wdir=foo['wdir'].values
    wspd=foo['wspd'].values
    wspdmax=foo['wspd_max'].values  
    raina=foo['raina_event'].values
    
    if (len(foo['pres'])==289):
        ## note: this only includes 0-288, not the midnight of the subsequent day which is value 289
        pres_all=np.append(pres_all,pres[0:288])  
        tdry_all=np.append(tdry_all,tdry[0:288])
        rh_all=np.append(rh_all,rh[0:288])
        wdir_all=np.append(wdir_all,wdir[0:288])
        wspd_all=np.append(wspd_all,wspd[0:288])
        wspdmax_all=np.append(wspdmax_all,wspdmax[0:288])
        raina_all=np.append(raina_all,raina[0:288])
        #print(pres_all.shape)
        #break
    
    if (len(pres)!=289):
        print(f'need to add missing values to {files[i]}')
        print(len(foo['pres']))
        count=count+1
        match_idx=[]
        ## calculate seconds since midnight
        timefoo=((foo['time'][:]-foo['time'][0]))//10**9  ## subtract first time, convert to seconds
        some_seconds=timefoo.values.astype(int)
        #print(some_seconds.shape)
        #print(f'length of some_seconds {some_seconds.shape}')
        #print(some_seconds)
        all_seconds=np.arange(0,86400+300,300)
        #print(f'length of all_seconds {all_seconds.shape}')
        #print(all_seconds)
        ### first - i find where the time arrays match
        match_idx=np.arange(all_seconds.shape[0])[np.in1d(all_seconds,some_seconds)]
        #print(match_idx.shape)
        #print(match_idx)
        ## initialize arrays with same dimensions as all_seconds with value of fill value (-999)
        some_seconds_withmissing=np.zeros(len(all_seconds))-999
        pres_withmissing=np.zeros(len(all_seconds))-999
        tdry_withmissing=np.zeros(len(all_seconds))-999
        rh_withmissing=np.zeros(len(all_seconds))-999       
        wdir_withmissing=np.zeros(len(all_seconds))-999  
        wspd_withmissing=np.zeros(len(all_seconds))-999  
        wspdmax_withmissing=np.zeros(len(all_seconds))-999
        raina_withmissing=np.zeros(len(all_seconds))-999        
        ## next fill in the some_seconds_withmissing arrays where there is a good value (i.e., at match_idx)
        some_seconds_withmissing[match_idx]=all_seconds[match_idx]
        #print(some_seconds_withmissing.shape)
        #print(all_seconds.shape)
        ## repeat with the variables - starting with pres
        #print(pres.shape)  ## only the non-fill values, size < 289
        #print(pres_withmissing.shape)  ##with the fill values, size=289
        pres_withmissing[match_idx]=pres
        #print(pres_withmissing)  ## check that the missing values are appearing
        #break
        tdry_withmissing[match_idx]=tdry
        rh_withmissing[match_idx]=rh   
        wdir_withmissing[match_idx]=wdir
        wspd_withmissing[match_idx]=wspd
        wspdmax_withmissing[match_idx]=wspdmax
        raina_withmissing[match_idx]=raina
        ## append array with missing values
        pres_all=np.append(pres_all,pres_withmissing[0:288])
        tdry_all=np.append(tdry_all,tdry_withmissing[0:288])
        rh_all=np.append(rh_all,rh_withmissing[0:288])
        wdir_all=np.append(wdir_all,wdir_withmissing[0:288])
        wspd_all=np.append(wspd_all,wspd_withmissing[0:288])
        wspdmax_all=np.append(wspdmax_all,wspdmax_withmissing[0:288])
        raina_all=np.append(raina_all,raina_withmissing[0:288])
        ### sanity check
        sanity="no"
        if sanity=="yes":## and files2021[i]=='mesa/mlab.20210522.nc':
            print("sanity check - will break afterwards")
            print(tdry_withmissing)
            print(tdry_withmissing.shape)
            #print(tdry[-289:])
            #print(tdry[-289:].shape)
            plt.plot(tdry[-289:],'.')
            plt.show()
            break


working on mesa/mlab.20210101.nc
working on mesa/mlab.20210102.nc
working on mesa/mlab.20210103.nc
working on mesa/mlab.20210104.nc
working on mesa/mlab.20210105.nc
working on mesa/mlab.20210106.nc
working on mesa/mlab.20210107.nc
need to add missing values to mesa/mlab.20210107.nc
261
working on mesa/mlab.20210108.nc
working on mesa/mlab.20210109.nc
working on mesa/mlab.20210110.nc
working on mesa/mlab.20210111.nc
working on mesa/mlab.20210112.nc
need to add missing values to mesa/mlab.20210112.nc
288
working on mesa/mlab.20210113.nc
need to add missing values to mesa/mlab.20210113.nc
288
working on mesa/mlab.20210114.nc
need to add missing values to mesa/mlab.20210114.nc
288
working on mesa/mlab.20210115.nc
working on mesa/mlab.20210116.nc
working on mesa/mlab.20210117.nc
working on mesa/mlab.20210118.nc
working on mesa/mlab.20210119.nc
working on mesa/mlab.20210120.nc
need to add missing values to mesa/mlab.20210120.nc
287
working on mesa/mlab.20210121.nc
need to add missing values 

working on mesa/mlab.20210811.nc
working on mesa/mlab.20210812.nc
working on mesa/mlab.20210813.nc
working on mesa/mlab.20210814.nc
working on mesa/mlab.20210815.nc
working on mesa/mlab.20210816.nc
working on mesa/mlab.20210817.nc
working on mesa/mlab.20210818.nc
working on mesa/mlab.20210819.nc
working on mesa/mlab.20210820.nc
working on mesa/mlab.20210821.nc
working on mesa/mlab.20210822.nc
working on mesa/mlab.20210823.nc
working on mesa/mlab.20210824.nc
working on mesa/mlab.20210825.nc
working on mesa/mlab.20210826.nc
working on mesa/mlab.20210827.nc
working on mesa/mlab.20210828.nc
need to add missing values to mesa/mlab.20210828.nc
182
working on mesa/mlab.20210829.nc
need to add missing values to mesa/mlab.20210829.nc
49
working on mesa/mlab.20210830.nc
working on mesa/mlab.20210831.nc
working on mesa/mlab.20210901.nc
need to add missing values to mesa/mlab.20210901.nc
288
working on mesa/mlab.20210902.nc
need to add missing values to mesa/mlab.20210902.nc
287
working on mesa/ml

In [6]:
print(f'Reminder Number of days with missing info: {count}') 
print('shape of all variables - should be ndays*288 (105120 for a year, not 105485!)')
print(pres_all.shape)
print(tdry_all.shape)
print(wdir_all.shape)
print(wspd_all.shape)
print(wspdmax_all.shape)
print(rain_accum_all.shape) 

Reminder Number of days with missing info: 33
shape of all variables - should be ndays*288 (105120 for a year, not 105485!)
(105120,)
(105120,)
(105120,)
(105120,)
(105120,)


NameError: name 'rain_accum_all' is not defined

In [ ]:
#### sanity plot the data to check it
plt.plot(tdry_all,'.')
plt.ylim(-30,40)
plt.xlim(40750,42000) ## this is a time period with missing data
plt.show()

plt.plot(tdry_all[288:],'.') ## this plots the last day (last 288 values)
plt.ylim(-30,40) 
plt.show()
#print(tdry_all[288:])  
#print(tdry_all[-288:].shape)

In [ ]:
#### more sanity plots the data to check it - plot the entire timeseries for all variables

plt.plot(pres_all,'.')
plt.ylim(780,825)
print(len(pres_all))
plt.show()

plt.plot(tdry_all,'.')
plt.ylim(-30,40)
plt.show()

plt.plot(rh_all,'.')
plt.ylim(0,100)
plt.show()

plt.plot(wdir_all,'.')
plt.ylim(0,360)
plt.show()

plt.plot(wspdmax_all,'.',color='r')
plt.plot(wspd_all,'.')
plt.ylim(0,50)
plt.show()

plt.plot(raina_all,'.')
plt.ylim(0,max(raina_all))
plt.show()

In [ ]:
## create time arrays that are useful 
## hours (fractional hours within a day) and day (julian day)

## create all the seconds in a day
n_seconds_day=24*60*60
n_seconds_5minute=5*60 ## number of seconds in the timestep
#all_seconds=np.arange(0,n_seconds_day+n_seconds_5minute,n_seconds_5minute)
all_seconds=np.arange(0,n_seconds_day,n_seconds_5minute)
print(all_seconds.shape)
print(all_seconds)

# create all the seconds in all of the days
day=np.ones(288)*0
seconds=all_seconds
for i in np.arange(1,len(files)):
    foo=np.ones(288)*i
    #print(foo)
    day=np.append(day,foo)
    #print(day)
    seconds=np.append(seconds,all_seconds)
    #print(seconds)

## convert to fractional hours
hours=seconds/(60*60.)

## convert to julian day
day=day+1

## check the shapes
print(day.shape)
print(hours.shape)

## check the first ten values
print(day[0:10])
print(hours[0:10])

## check the last ten values
print(day[-10:])
print(hours[-10:])

In [ ]:
## check the shapes - should all be the same
print(day.shape)
print(hours.shape)
print(pres_all.shape)
print(tdry_all.shape)
print(wdir_all.shape)
print(wspd_all.shape)
print(wspdmax_all.shape)
print(raina_all.shape)

## check the first ten values
print(day[0:10])
print(hours[0:10])
print(pres_all[0:10])
print(tdry_all[0:10])
print(wdir_all[0:10])
print(wspd_all[0:10])
print(wspdmax_all[0:10])
print(raina_all[0:10])

## check the last ten values
print(day[-10:])
print(hours[-10:])
print(pres_all[-10:])
print(tdry_all[-10:])
print(wdir_all[-10:])
print(wspd_all[-10:])
print(wspdmax_all[-10:])
print(raina_all[-10:])

In [ ]:
#### Save it out as a netcdf file

## 1d data (day)
data_vars={'days': (['day'],       day,{'units': 'days_foolingxarray'}),\
          'hour_frac': (['day'],  hours,{'units': 'hours_in_day_since_00_UTC'}),\
          'pres': (['day'],       pres_all,{'units': 'millibars'},{'_FillValue': '-999'}),\
          'tdry': (['day'],       tdry_all,{'units': 'deg_C'},{'_FillValue': '-999'}),\
          'rh': (['day'],         rh_all,{'units': 'percent'},{'_FillValue': '-999'}),\
          'wdir': (['day'],       wdir_all,{'units': 'degrees'},{'_FillValue': '-999'}),\
          'wspd': (['day'],       wspd_all,{'units': 'meters_per_second'},{'_FillValue': '-999'}),\
          'wspdmax': (['day'],    wspdmax_all,{'units': 'meters_per_second'},{'_FillValue': '-999'}),\
          'raina_event': (['day'], raina_all,{'units': 'millimeters'},{'_FillValue': '-999'}),\
          }
ds = xr.Dataset(data_vars,coords={'day':(['day'],day)})
ds.attrs['site']='Mesa Lab NCAR 5-minute (360 second) weather data'
ds.attrs['data_from']='ftp://ftp.eol.ucar.edu/pub/archive/weather'
ds.attrs['processed_date']=np.str(date.today())
ds.attrs['contact']='Jennifer.E.Kay@colorado.edu'
fname='mesalab_2021_data.nc'
ds.to_netcdf(fname)

In [ ]:
print(wspdmax_all[-1:])
print(wspd_all[-1:])
print(wspdmax_all[1])
print(wspd_all[1])

In [ ]:
ds=xr.open_dataset('mesalab_2021_data.nc',decode_times=False)
df = ds.to_dataframe()

## preview data (also through df.head() & df.tail())
df

In [ ]:
df.days.nunique()

In [ ]:
#### Save data frame to csv file

df.to_csv('mesalab_2021_data.csv', index=False)